In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install langchain-huggingface

import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from pyspark.sql import SparkSession, DataFrame

from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import functions as F
from pyspark.sql import types as T

from pyspark.ml.feature import StringIndexer, Imputer
from pyspark.ml.feature import VectorAssembler, ChiSqSelector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 3s (127 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Sk

In [ ]:
spark = SparkSession.builder \
               .appName('Tarea_3') \
               .getOrCreate()

In [ ]:
spark

In [ ]:
#Se especifica la ubicación del archivo donde se tienen los datos
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#Se abre el archivo
%cd /content/drive/My Drive/
#df = pd.read_csv('events.csv')
df = spark.read.csv(path = '/content/drive/My Drive/events.csv', header = True)

/content/drive/My Drive


In [ ]:
#Se imprime el schema de los datos
df.printSchema()

root
 |-- id_odsp: string (nullable = true)
 |-- id_event: string (nullable = true)
 |-- sort_order: string (nullable = true)
 |-- time: string (nullable = true)
 |-- text: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_type2: string (nullable = true)
 |-- side: string (nullable = true)
 |-- event_team: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- player: string (nullable = true)
 |-- player2: string (nullable = true)
 |-- player_in: string (nullable = true)
 |-- player_out: string (nullable = true)
 |-- shot_place: string (nullable = true)
 |-- shot_outcome: string (nullable = true)
 |-- is_goal: string (nullable = true)
 |-- location: string (nullable = true)
 |-- bodypart: string (nullable = true)
 |-- assist_method: string (nullable = true)
 |-- situation: string (nullable = true)
 |-- fast_break: string (nullable = true)



In [ ]:
#Se guardan los nombres de las columnas en una variable
columns = df.columns
columns

['id_odsp',
 'id_event',
 'sort_order',
 'time',
 'text',
 'event_type',
 'event_type2',
 'side',
 'event_team',
 'opponent',
 'player',
 'player2',
 'player_in',
 'player_out',
 'shot_place',
 'shot_outcome',
 'is_goal',
 'location',
 'bodypart',
 'assist_method',
 'situation',
 'fast_break']

In [ ]:
#Se hace una variable con las columnas que son de tipo string para quitarlas de la variable de las columnas
string_columns = ['id_odsp', 'id_event', 'text', 'event_team', 'opponent', 'player', 'player2', 'player_in', 'player_out']
for column in string_columns:
  columns.remove(column)

columns

['sort_order',
 'time',
 'event_type',
 'event_type2',
 'side',
 'shot_place',
 'shot_outcome',
 'is_goal',
 'location',
 'bodypart',
 'assist_method',
 'situation',
 'fast_break']

In [ ]:
#Se cambian las variables que son numericas a tipo int
for column in columns:
  df = df.withColumn(column, F.col(column).cast(T.IntegerType()))

In [ ]:
#Se imprime el schema con los nuevos tipos de variables.
df.printSchema()

root
 |-- id_odsp: string (nullable = true)
 |-- id_event: string (nullable = true)
 |-- sort_order: integer (nullable = true)
 |-- time: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- event_type2: integer (nullable = true)
 |-- side: integer (nullable = true)
 |-- event_team: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- player: string (nullable = true)
 |-- player2: string (nullable = true)
 |-- player_in: string (nullable = true)
 |-- player_out: string (nullable = true)
 |-- shot_place: integer (nullable = true)
 |-- shot_outcome: integer (nullable = true)
 |-- is_goal: integer (nullable = true)
 |-- location: integer (nullable = true)
 |-- bodypart: integer (nullable = true)
 |-- assist_method: integer (nullable = true)
 |-- situation: integer (nullable = true)
 |-- fast_break: integer (nullable = true)



In [ ]:
df = df.dropna()

In [ ]:
#Se pasan los valores en texto a una categoria

column_a_indexar = ['event_team', 'opponent', 'player', 'player2', 'player_in', 'player_out']
for col in column_a_indexar:
  indexer = StringIndexer(inputCol = col, outputCol = col + '_indexada')
  df = indexer.fit(df).transform(df)

In [ ]:
df.show()

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+------------------+--------------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------------------+-----------------+---------------+----------------+------------------+-------------------+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|            player|             player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|event_team_indexada|opponent_indexada|player_indexada|player2_indexada|player_in_indexada|player_out_indexada|
+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+------------------+--------------------+---------+----------+----------+------------+-------+--------+--------+------

In [ ]:
#Se dejan solo aquellos eventos que pertenecen a un gol

df_ml = df.filter(df.is_goal == 1).select('is_goal', 'time', 'event_team', 'event_team_indexada', 'opponent', 'opponent_indexada', 'player', 'player_indexada', 'location', 'bodypart', 'situation')

In [ ]:
df_ml.show()

+-------+----+--------------------+-------------------+-------------------+-----------------+------------------+---------------+--------+--------+---------+
|is_goal|time|          event_team|event_team_indexada|           opponent|opponent_indexada|            player|player_indexada|location|bodypart|situation|
+-------+----+--------------------+-------------------+-------------------+-----------------+------------------+---------------+--------+--------+---------+
|      1|  17|   Borussia Dortmund|               10.0|         Hamburg SV|              7.0| kevin grosskreutz|          564.0|       9|       2|        1|
|      1|  29|   Borussia Dortmund|               10.0|         Hamburg SV|              7.0|       mario gotze|          217.0|       3|       1|        1|
|      1|  48|         SC Freiburg|               72.0|        FC Augsburg|             21.0|papiss demba cisse|          280.0|      15|       1|        1|
|      1|  53|         FC Augsburg|               38.0|   

In [ ]:
#se haran 2 modelos de machine learning, uno para determinar con que parte del cuerpo fue el gol y otra para determnar la situación
#Se generan 2 variables de inputs
inputs_bodypart = ['time', 'event_team_indexada', 'opponent_indexada', 'location', 'situation']
inputs_situation = ['time', 'location', 'bodypart']

In [ ]:
#Se genera la columna de features para la parte de cuerpo
assembler = VectorAssembler(inputCols = inputs_bodypart, outputCol = "features")
df_ml_bodypart = assembler.transform(df_ml)

df_ml_bodypart.select('features').show()

+--------------------+
|            features|
+--------------------+
|[17.0,10.0,7.0,9....|
|[29.0,10.0,7.0,3....|
|[48.0,72.0,21.0,1...|
|[53.0,38.0,64.0,3...|
|[55.0,72.0,21.0,3...|
|[81.0,38.0,64.0,3...|
|[81.0,41.0,131.0,...|
|[28.0,22.0,45.0,1...|
|[34.0,78.0,86.0,1...|
|[80.0,93.0,57.0,3...|
|[20.0,74.0,107.0,...|
|[45.0,35.0,74.0,3...|
|[73.0,35.0,74.0,1...|
|[6.0,42.0,47.0,3....|
|[10.0,16.0,22.0,3...|
|[90.0,18.0,73.0,3...|
|[45.0,128.0,34.0,...|
|[75.0,20.0,135.0,...|
|[37.0,52.0,26.0,1...|
|[56.0,52.0,26.0,3...|
+--------------------+
only showing top 20 rows



In [ ]:
#Se genera la columna de features para la situación
assembler_2 = VectorAssembler(inputCols = inputs_situation, outputCol = "features")
df_ml_situation = assembler_2.transform(df_ml)

df_ml_situation.select('features').show()

+---------------+
|       features|
+---------------+
| [17.0,9.0,2.0]|
| [29.0,3.0,1.0]|
|[48.0,15.0,1.0]|
| [53.0,3.0,3.0]|
| [55.0,3.0,3.0]|
| [81.0,3.0,3.0]|
|[81.0,13.0,3.0]|
|[28.0,10.0,1.0]|
|[34.0,15.0,1.0]|
| [80.0,3.0,2.0]|
|[20.0,11.0,1.0]|
| [45.0,3.0,3.0]|
|[73.0,15.0,1.0]|
|  [6.0,3.0,3.0]|
| [10.0,3.0,1.0]|
| [90.0,3.0,1.0]|
|[45.0,15.0,2.0]|
|[75.0,11.0,2.0]|
|[37.0,13.0,3.0]|
| [56.0,3.0,3.0]|
+---------------+
only showing top 20 rows



In [ ]:
# Selección de características usando Chi-Square para la parte del cuerpo
selector = ChiSqSelector(numTopFeatures = 3, featuresCol = "features", labelCol="bodypart", outputCol="selected_features")
df_ml_bodypart_2 = selector.fit(df_ml_bodypart).transform(df_ml_bodypart)
df_ml_bodypart_2.show(5)

+-------+----+-----------------+-------------------+-----------+-----------------+------------------+---------------+--------+--------+---------+--------------------+-----------------+
|is_goal|time|       event_team|event_team_indexada|   opponent|opponent_indexada|            player|player_indexada|location|bodypart|situation|            features|selected_features|
+-------+----+-----------------+-------------------+-----------+-----------------+------------------+---------------+--------+--------+---------+--------------------+-----------------+
|      1|  17|Borussia Dortmund|               10.0| Hamburg SV|              7.0| kevin grosskreutz|          564.0|       9|       2|        1|[17.0,10.0,7.0,9....|   [10.0,9.0,1.0]|
|      1|  29|Borussia Dortmund|               10.0| Hamburg SV|              7.0|       mario gotze|          217.0|       3|       1|        1|[29.0,10.0,7.0,3....|   [10.0,3.0,1.0]|
|      1|  48|      SC Freiburg|               72.0|FC Augsburg|           

In [ ]:
#Se escalan las features seleccionadas
scaler = StandardScaler(inputCol= 'selected_features', outputCol= 'features_scaled')
df_ml_bodypart_2_scaled = scaler.fit(df_ml_bodypart_2).transform(df_ml_bodypart_2)

df_ml_bodypart_2_scaled.show()

+-------+----+--------------------+-------------------+-------------------+-----------------+------------------+---------------+--------+--------+---------+--------------------+-----------------+--------------------+
|is_goal|time|          event_team|event_team_indexada|           opponent|opponent_indexada|            player|player_indexada|location|bodypart|situation|            features|selected_features|     features_scaled|
+-------+----+--------------------+-------------------+-------------------+-----------------+------------------+---------------+--------+--------+---------+--------------------+-----------------+--------------------+
|      1|  17|   Borussia Dortmund|               10.0|         Hamburg SV|              7.0| kevin grosskreutz|          564.0|       9|       2|        1|[17.0,10.0,7.0,9....|   [10.0,9.0,1.0]|[0.29579183596684...|
|      1|  29|   Borussia Dortmund|               10.0|         Hamburg SV|              7.0|       mario gotze|          217.0|    

In [ ]:
# Dividir datos en entrenamiento y prueba
train_bodypart, test_bodypart = df_ml_bodypart_2_scaled.randomSplit([0.7, 0.3])

# Train the model
rf = RandomForestClassifier(featuresCol="features_scaled", labelCol="bodypart")
model = rf.fit(train_bodypart)

# Predictions and evaluation
predictions = model.transform(test_bodypart)
evaluator = MulticlassClassificationEvaluator(labelCol="bodypart", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

accuracy

0.5850041771094403

In [ ]:
# Dividir datos en entrenamiento y prueba
train_situation, test_situation = df_ml_situation.randomSplit([0.7, 0.3])

# Train the model
rf_2 = RandomForestClassifier(featuresCol="features", labelCol="situation")
model_2 = rf_2.fit(train_situation)

# Predictions and evaluation
predictions_2 = model_2.transform(test_situation)
evaluator_2 = MulticlassClassificationEvaluator(labelCol="situation", predictionCol="prediction",
                                              metricName="accuracy")
accuracy_2 = evaluator_2.evaluate(predictions_2)

accuracy_2

0.8119856509812197

In [ ]:
scaler_2 = StandardScaler(inputCol= 'features', outputCol= 'features_scaled')
df_ml_situation_scaled = scaler_2.fit(df_ml_situation).transform(df_ml_situation)

df_ml_situation_scaled.show()

+-------+----+--------------------+-------------------+-------------------+-----------------+------------------+---------------+--------+--------+---------+---------------+--------------------+
|is_goal|time|          event_team|event_team_indexada|           opponent|opponent_indexada|            player|player_indexada|location|bodypart|situation|       features|     features_scaled|
+-------+----+--------------------+-------------------+-------------------+-----------------+------------------+---------------+--------+--------+---------+---------------+--------------------+
|      1|  17|   Borussia Dortmund|               10.0|         Hamburg SV|              7.0| kevin grosskreutz|          564.0|       9|       2|        1| [17.0,9.0,2.0]|[0.64604680125250...|
|      1|  29|   Borussia Dortmund|               10.0|         Hamburg SV|              7.0|       mario gotze|          217.0|       3|       1|        1| [29.0,3.0,1.0]|[1.10207983743074...|
|      1|  48|         SC Frei

In [ ]:
# Dividir datos en entrenamiento y prueba
train_situation, test_situation = df_ml_situation_scaled.randomSplit([0.7, 0.3])

# Train the model
rf_2 = RandomForestClassifier(featuresCol="features_scaled", labelCol="situation")
model_2 = rf_2.fit(train_situation)

# Predictions and evaluation
predictions_2 = model_2.transform(test_situation)
evaluator_2 = MulticlassClassificationEvaluator(labelCol="situation", predictionCol="prediction",
                                              metricName="accuracy")
accuracy_2 = evaluator_2.evaluate(predictions_2)

accuracy_2

0.8088358458961474